# Data Encoder
Uses an autoencoder to reduce the dimensionality of the data.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 15:45:16.116636: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 15:45:16.660845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 15:45:17.307602: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:45:17.331361: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:45:17.331408: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The data should already be present as `dataset.csv` and `top_unigrams.txt` in the `data` folder.

If they are not present, do the following.
1. Ensure that the VirusTotal reports are present in `data/json` with the format `[LABEL]_[HASH].json`.
3. Run `prepare_data.py`. This will generate the two files needed for this notebook.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/dataset.csv")
raw_df

,label,hash,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
2,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COBALTSTRIKE,dab956e9c864a84d12e8106a24ac3cf2950394152c62b6...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,UNKNOWN,c40861e5ebd3c30de810f33c0959aaf5683586fe819998...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1569,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1570,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1571,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the training of the model, we don't need the label or the file hash.

In [5]:
df = raw_df.drop(columns=["label", "hash"])
df

,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,dim-0009,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1570,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


80% of the dataframe will be saved for training, while 20% will be left for testing.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(
    df, test_size=0.2, random_state=42
)

In [7]:
X_train.shape

(1258, 10000)

In [8]:
X_test.shape

(315, 10000)

# Model Creation

We will use an autoencoder to reduce the dimensionality of the data.

In [9]:
from keras import layers
from keras.models import Sequential

In [10]:
LAYER_SIZES = [512, 256, 128, 32]  # The last layer is the center layer

In [11]:
INITIAL_EPOCH = 0

def create_encoder():
    model = Sequential(name="Encoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))

    for layer_size in LAYER_SIZES[:-1]:
        model.add(layers.Dense(layer_size, activation="relu"))

    # Add an activity regularizer to make the middle layer sparse
    model.add(layers.Dense(LAYER_SIZES[-1], activation="relu"))
    
    return model


def create_decoder():
    model = Sequential(name="Decoder")
    model.add(keras.Input((LAYER_SIZES[-1],), name="decoder-input"))

    for layer_size in LAYER_SIZES[-2::-1]:  # Starting from second last
        model.add(layers.Dense(layer_size, activation="relu"))
    model.add(layers.Dense(df.shape[1], activation="relu"))
    
    return model


def create_autoencoder(encoder, decoder):
    model = Sequential(name="Autoencoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))
    model.add(encoder)
    model.add(decoder)

    model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"]
    )

    return model

In [12]:
encoder = create_encoder()
decoder = create_decoder()
autoencoder = create_autoencoder(encoder, decoder)

2024-04-29 15:45:18.998114: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:45:18.998206: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:45:18.998238: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:45:19.713066: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:45:19.713254: I external/local_xla/xla/stream_executor

In [13]:
autoencoder.summary(expand_nested=True)

Model: "Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 32)             │     5,288,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 10000)          │     5,298,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_4 (Dense)            │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_5 (Dense)            │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_6 (Dense)            │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_7 (Dense)            │ (None, 10000)          │     5,130,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,587,696 (40.39 MB)

 Trainable params: 10,587,696 (40.39 MB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [14]:
import os

checkpoint_path = "../models/encoder/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [15]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [16]:
# INITIAL_EPOCH = 54
# autoencoder = keras.models.load_model(checkpoint_path)

Train the model.

In [17]:
NUM_EPOCHS = 200

autoencoder.fit(
    X_train,
    X_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/200


I0000 00:00:1714376721.648620 1190698 service.cc:145] XLA service 0x7f0624005fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714376721.648681 1190698 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 15:45:21.680168: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 15:45:22.582892: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


16/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0186 - mae: 0.0264

I0000 00:00:1714376725.084993 1190698 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0178 - mae: 0.0259
Epoch 1: val_loss improved from inf to 0.01451, saving model to ../models/encoder/checkpoint.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 138ms/step - loss: 0.0177 - mae: 0.0258 - val_loss: 0.0145 - val_mae: 0.0209
Epoch 2/200
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0135 - mae: 0.0198
Epoch 2: val_loss improved from 0.01451 to 0.01269, saving model to ../models/encoder/checkpoint.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0135 - mae: 0.0197 - val_loss: 0.0127 - val_mae: 0.0185
Epoch 3/200
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0115 - mae: 0.0168
Epoch 3: val_loss improved from 0.01269 to 0.01152, saving model to ../models/encoder/checkpoint.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0115 - mae: 0.0167 - val_loss: 0.0115 - val_mae: 0.0163
Epoch 4/200
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0108 - mae: 0.0152
Epoch 4: val_loss improved from 0.01152 to 0.01081, saving model

Load the best performing model.

In [18]:
autoencoder = keras.models.load_model(checkpoint_path)

Evaluate the autoencoder on the test data.

In [19]:
test_mse, test_mae = autoencoder.evaluate(X_test, X_test, verbose=1)
print(f"Testing MSE: {test_mse:5.5f}")
print(f"Testing MAE: {test_mae:5.5f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.0070 - mae: 0.0089
Testing MSE: 0.00699
Testing MAE: 0.00882


Get only the encoder part to save.

In [20]:
encoder = autoencoder.get_layer("Encoder")

In [21]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,288,864 (20.18 MB)

 Trainable params: 5,288,864 (20.18 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
encoder.save("../models/encoder/encoder.keras")

# Transforming Original Data

In [23]:
transformed_df = encoder.predict(df)

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [24]:
transformed_df = pd.DataFrame(transformed_df, columns=[f"dim-{i:02d}" for i in range(LAYER_SIZES[-1])])

Add the labels and hashes back to the dataframe.

In [25]:
transformed_df.insert(0, "label", raw_df["label"])
transformed_df.insert(1, "hash", raw_df["hash"])

In [26]:
transformed_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,7.549576,0.000000,0.0,8.502934,24.176189,15.322744,6.537903,17.014332,...,3.867043,0.0,2.907426,0.000000,0.000000,14.940077,0.000000,0.0,0.0,0.000000
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.000000,0.000000,0.0,14.619308,0.000000,0.000000,16.847561,7.412828,...,12.765860,0.0,0.000000,9.257461,12.005179,13.770399,12.168733,0.0,0.0,8.010514
2,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,13.003604,0.000000,0.0,16.939367,21.694796,11.356659,6.433365,16.684380,...,18.388371,0.0,3.366971,0.000000,0.000000,22.011396,0.000000,0.0,0.0,0.000000
3,COBALTSTRIKE,dab956e9c864a84d12e8106a24ac3cf2950394152c62b6...,9.411519,11.802638,0.0,8.384830,0.000000,7.831763,9.012511,0.000000,...,0.000000,0.0,0.000000,7.769751,0.000000,4.577077,0.000000,0.0,0.0,8.902426
4,UNKNOWN,c40861e5ebd3c30de810f33c0959aaf5683586fe819998...,4.599778,4.875384,0.0,2.622899,0.000000,12.233712,3.769575,5.864996,...,3.280627,0.0,1.568317,0.744919,0.000000,4.647032,0.000000,0.0,0.0,6.844474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,0.000000,0.000000,0.0,12.429248,0.000000,13.549780,15.505765,13.514245,...,20.763781,0.0,0.000000,0.000000,6.163751,0.922269,0.000000,0.0,0.0,1.915500
1569,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,20.171080,4.978172,0.0,0.000000,3.767549,13.185392,17.561668,4.622786,...,0.373729,0.0,0.000000,6.819319,0.000000,5.469461,0.000000,0.0,0.0,0.000000
1570,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,7.412106,3.795544,0.0,3.142282,0.000000,8.510063,9.629321,6.728159,...,16.329901,0.0,6.062054,8.711801,12.110841,13.719257,0.000000,0.0,0.0,10.587579
1571,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,5.062665,3.779000,0.0,0.000000,0.000000,11.753929,0.457823,7.567391,...,6.432997,0.0,2.620044,1.316070,0.000000,3.945687,0.000000,0.0,0.0,4.418612


In [27]:
transformed_df.to_csv("../data/encoded-data.csv", index=False)